## Label users

0 - non-commuter, 1 - commuter

In [1]:
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import pickle
import os

### Step1: Extrac Users' Trips

In [19]:
def metroUsersAnalysis(): #  Gen User Trip
    #inFile = 'results/metroData_sorted.csv'
    inFile = '/data6/peiyan/SH-METR/results/metroData_sortedByUser.csv'
    userData = open(inFile, 'r')

    #outFile = 'results/metroData_filtered.csv'
    outFile = '/data6/peiyan/SH-METR/results/metroData_userTrip.csv'
    outData = open(outFile, 'w')

    #thres = 100
    thres = 0
    userCount = 0
    userCount_filter = 0
    # userTrips = {}
    numRecords = []

    preUser = ''
    trips = []

    for line in userData:
        line = line.rstrip().split(',')
        currentUser = int(line[0])
        currentDay = line[1]
        StartTime = line[2]
        EndTime = line[3]
        inStation = line[4]
        outStation = line[5]

        if inStation == '' or outStation == '':
            continue

        # counting the trips of each user
        if preUser == '':
            preUser = currentUser

        if currentUser != preUser:
            # process last user
            # filter users
            if len(trips) >= thres:
                # userTrips[preUser] = trips
                userCount_filter += 1
                # save data
                outData.writelines(str(preUser) + '\n')
                for trip in trips:
                    outData.writelines(','.join(trip) + '\n')
            numRecords.append(len(trips))

            # initialize
            preUser = currentUser
            trips = []
            userCount += 1
            if userCount % 1e6 == 0:
                print ('Processs userNum:'+str(userCount))

        trips.append([currentDay, StartTime, EndTime, inStation, outStation])

    userData.close()
    outData.close()

    with open('/data6/peiyan/SH-METR/results/numRecords.pkl','wb') as outfile:
        pickle.dump(numRecords,outfile)

    return numRecords


In [15]:
# 去除 header
!tail -n +2 '/data/peiyan/Research/SH-Metr/Metro/metroData_ts_v2.csv'>'/data6/peiyan/SH-METR/results/metroData_ts_v2.csv'


In [17]:
# inFile = "/media/xu/OS/Data/SmartCardData/metroData_selected.csv"
# outFile = 'results/metroData_sorted.csv'
inFile = '/data6/peiyan/SH-METR/results/metroData_ts_v2.csv'
outFile = '/data6/peiyan/SH-METR/results/metroData_sortedByUser.csv' 
#os.system('sort --header -t, -k1,1n -k3,3n -k4,4n ' + inFile + ' > ' + outFile) # sorted by user and by time
os.system('sort -t, -k1,1n -k4,4n -k5,5n ' + inFile + ' > ' + outFile) # Start time is added as a new col

0

In [20]:
numRecords = metroUsersAnalysis() # extract trips for each user

Processs userNum:1000000
Processs userNum:2000000
Processs userNum:3000000
Processs userNum:4000000
Processs userNum:5000000
Processs userNum:6000000
Processs userNum:7000000
Processs userNum:8000000
Processs userNum:9000000
Processs userNum:10000000
Processs userNum:11000000
Processs userNum:12000000
Processs userNum:13000000
Processs userNum:14000000
Processs userNum:15000000
Processs userNum:16000000
Processs userNum:17000000
Processs userNum:18000000
Processs userNum:19000000


In [21]:
print('userNum:'+str(len(numRecords)))
# plot the distribution of numRecords
interval = 5
bins = np.linspace(0, 200, 41)
usagesHist = np.histogram(np.array(numRecords), bins)
bins = np.array(bins[1:])
usagesHist = np.divide(usagesHist[0], float(np.sum(usagesHist[0])))
print (usagesHist)

fig = plt.figure(figsize=(6, 4))
ax = plt.subplot(1, 1, 1)
plt.bar(bins.tolist(), usagesHist.tolist(), align='edge', width=interval, linewidth=1, facecolor='#41A7D8',
        edgecolor='k',
        label='data')

plt.xlim(0, 30)
# plt.ylim(0, 0.05)
plt.xticks(range(0, 200, 10))
# plt.yscale("log")
plt.xlabel(r'# of trips', fontsize=12)
plt.ylabel(r"Fraction", fontsize=12)

plt.tight_layout()
plt.savefig('results/metro_trip_distribution.png', dpi=150)
# plt.savefig('results/metro_trip_distribution.pdf')
plt.close() 

userNum:19695580
[0.23656925 0.18901107 0.11548176 0.07700859 0.05509985 0.04119176
 0.03234499 0.02605885 0.02159262 0.01815667 0.01558566 0.01352798
 0.01187111 0.01059053 0.00948929 0.00864993 0.00788446 0.0072594
 0.00673522 0.00630862 0.00584198 0.00554415 0.00521854 0.00497547
 0.00472497 0.00455239 0.00436996 0.00420681 0.0041009  0.00401945
 0.0040194  0.00414597 0.00436325 0.00447744 0.00449201 0.00439475
 0.00420335 0.00398686 0.00375857 0.00418616]


***
### Step 2: Label users

In [2]:
# label user as commuter and non-commuter with temporal information
# 0 - non-commuter, 1 - commuter
# label the visited stations as home (h), work (w), other (o)
# label user's trip as home-work (0), home-other (1), work-other (2), other-other (3)
def userLabel(trips):
    # select in station in the morning (6-10) and out station in the evening (5-23)
    homeStationsPool = []
    workStationsPool = []

    visitedStations = set()
    visitedTrips = set()
    for trip in trips:
        transHour = int(trip[1][:2])
        inStation = trip[2]
        outStation = trip[3]
        if transHour <= 10:
            homeStationsPool.append(inStation)
            workStationsPool.append(outStation)
        if transHour >= 17:
            homeStationsPool.append(outStation)
            workStationsPool.append(inStation)

        visitedStations.add(inStation)
        visitedStations.add(outStation)
        visitedTrips.add((inStation, outStation))

    homeFrequency = Counter(homeStationsPool) # return dict
    workFrequency = Counter(workStationsPool)

    # select the station as home or work if there is one station has frequency larger than 40%

    for station in homeFrequency:
        homeFrequency[station] = homeFrequency[station]/float(len(homeStationsPool))

    for station in workFrequency:
        workFrequency[station] = workFrequency[station] / float(len(workStationsPool))

    # sortedHomes = sorted(homeFrequency.iteritems(), key=lambda (k,v): (v,k), reverse=True)  # list of (key, value)
    # sortedWorks = sorted(workFrequency.iteritems(), key=lambda (k,v): (v,k), reverse=True)  # list of (key, value)

    sortedHomes = sorted(homeFrequency.items(), key=lambda x: (x[1], x[0]), reverse=True)
    sortedWorks = sorted(workFrequency.items(), key=lambda x: (x[1], x[0]), reverse=True) # sorted by values; then by keys

    homeStation = 0 
    workStation = 0
    if len(sortedHomes)>0:
        if sortedHomes[0][1] > 0.4: # [0] indicates highest prob
            homeStation = sortedHomes[0][0]
    if len(sortedWorks)>0:
        if sortedWorks[0][1] > 0.4:
            workStation = sortedWorks[0][0]

    commuterLabel = -1
    if homeStation==0:
        # remove the user
        return commuterLabel, {}, {}, {'h':-1, 'w':-1}

    if workStation == 0:  
        # non-commuters: no work station
        commuterLabel = 0
    else:
        commuterLabel = 1

    stationLables_light = {}
    stationLables_light['h'] = homeStation
    stationLables_light['w'] = -1  if workStation==0 else workStation

    # label stations
    stationLables = {}
    tripLabels = {}
    for station in visitedStations:
        if station == homeStation:
            stationLables[station] = 'h'
        elif station == workStation:
            stationLables[station] = 'w'
        else:
            stationLables[station] = 'o'
    for trip in visitedTrips:
        inLabel = stationLables[trip[0]]
        outLabel = stationLables[trip[1]]
        if (inLabel, outLabel) == ('h', 'w') or (inLabel, outLabel) == ('w', 'h'):
            tripLabels[trip] = 0
            continue
        if (inLabel, outLabel) == ('h', 'o') or (inLabel, outLabel) == ('o', 'h'):
            tripLabels[trip] = 1
            continue
        if (inLabel, outLabel) == ('w', 'o') or (inLabel, outLabel) == ('o', 'w'):
            tripLabels[trip] = 2
            continue
        tripLabels[trip] = 3

    return commuterLabel, stationLables, tripLabels, stationLables_light


# label the user to commuter and non-commuter
def userTripLabel():
    
    userLabels = {}
    stationLabels = {}
    tripLabels = {}
    station_lightLabels = {}
    # load the metro stations
    #stationData = open("results/metroStations.csv", 'r')
    stationData = open("../Metro/station_transInfo_cleaned.csv")
    stationLoc = {}
    next(stationData)
    for row in stationData:
        row = row.rstrip().split(',')
        # stationID = int(row[0])
        # lon = float(row[1])
        # lat = float(row[2])
        stationID = int(row[1])
        lon = float(row[2])
        lat = float(row[3])
        stationLoc[stationID] = (lon, lat)

    # new tranfer station ids
    transferStation, transferStationToNewID = pickle.load(open("../Metro/transferStations.pkl", 'rb'))

    # inFile = 'results/metroData_filtered.csv'
    inFile = '/data6/peiyan/SH-METR/results/metroData_userTrip.csv'
    userData = open(inFile, 'r')

    # ==== start of reading trips =====

    userCount = 0
    userCount_filter = 0 # not used 
    userTrips = {}
    numRecords = []

    preUser = ''
    trips = []

    for line in userData:
        line = line.rstrip().split(',')
        if len(line) == 1:
            # a new user
            currentUser = int(line[0])
            if preUser == '':
                preUser = currentUser

            # process the last user
            if currentUser != preUser:
                # filter users
                numRecords.append(len(trips))
                userTrips[preUser] = trips
                # ================================
                # Analyze the trips
                # ================================
                userL, stationL, tripL, stationL_light = userLabel(trips)
                userLabels[preUser] = userL  # int
                stationLabels[preUser] = stationL # dict
                tripLabels[preUser] = tripL  # dict
                station_lightLabels[preUser] = stationL_light #dict

                # initialize
                preUser = currentUser
                trips = []
                userCount += 1
                if userCount % 1e6 == 0:
                    print ('Processs userNum:'+str(userCount))

            continue

        currentDay = line[0]
        StartTime = line[1]
        EndTime = line[2]
        inStation = int(line[3])
        outStation = int(line[4])

        trips.append([currentDay, EndTime, inStation, outStation])  # with updated stationName

    # process the last user
    userL, stationL, tripL, stationL_light = userLabel(trips)
    userLabels[preUser] = userL
    stationLabels[preUser] = stationL
    tripLabels[preUser] = tripL
    station_lightLabels[preUser] = stationL_light

    # save
    #pickle.dump([userLabels, stationLabels, tripLabels], open("/data6/peiyan/SH-METR/results/userLabels.pkl", 'wb'), pickle.HIGHEST_PROTOCOL)
    print('begin Saving files')

    with open('/data6/peiyan/SH-METR/results/userLabels.pkl','wb') as outfile:
        pickle.dump(userLabels, outfile)

    with open('/data6/peiyan/SH-METR/results/stationLabels.pkl','wb') as outfile:
        pickle.dump(stationLabels, outfile)

    with open('/data6/peiyan/SH-METR/results/TripLabels.pkl','wb') as outfile:
        pickle.dump(tripLabels, outfile)

    with open('/data6/peiyan/SH-METR/results/station_lightLabels.pkl','wb') as outfile:
        pickle.dump(station_lightLabels, outfile)

    return userLabels, stationLabels, tripLabels, station_lightLabels



In [3]:
userLabels, stationLabels, tripLabels, station_lightLabels = userTripLabel()

Processs userNum:1000000
Processs userNum:2000000
Processs userNum:3000000
Processs userNum:4000000
Processs userNum:5000000
Processs userNum:6000000
Processs userNum:7000000
Processs userNum:8000000
Processs userNum:9000000
Processs userNum:10000000
Processs userNum:11000000
Processs userNum:12000000
Processs userNum:13000000
Processs userNum:14000000
Processs userNum:15000000
Processs userNum:16000000
Processs userNum:17000000
Processs userNum:18000000
Processs userNum:19000000
begin Saving files


In [29]:
numCommuters = np.sum([i for i in userLabels.values() if i==1])
numNoHome = np.sum([-i for i in userLabels.values() if i==-1])
numNonCommuters = len(userLabels) - numCommuters - numNoHome

print( "# of commuters : {}".format(numCommuters/float(numNonCommuters + numCommuters)))
print("# of non-commuters : {}".format(numNonCommuters/float(numNonCommuters + numCommuters)))
print ("# of no home : {}".format(numNoHome/float(len(userLabels))))

# distribution of trip distances for all, commuters, non-commuters
# distances = pickle.load(open("results/stationDistances.pkl", 'rb'))

# allTripsDistances = []
# distancesInUserTypes = [[] for i in range(2)]
# distancesInTripTypes = [[] for i in range(4)]

# for user in userLabels:
#     label = userLabels[user]
#     for trip, tripL in tripLabels[user].iteritems():
#         dist = distances[trip]
#         allTripsDistances.append(dist)
#         distancesInUserTypes[label].append(dist)
#         distancesInTripTypes[tripL].append(dist)

# # plot the distritbuion
# print (np.max(allTripsDistances))

# # travel distance of varying types of users
# numInterval = 100
# maxV = 100.0
# interval = maxV / numInterval
# bins = np.linspace(0, maxV, numInterval + 1)


# usagesHist = np.histogram(np.array(allTripsDistances), bins)
# usagesHist_all = np.divide(usagesHist[0], float(np.sum(usagesHist[0])))

# # non-commuter
# usagesHist = np.histogram(np.array(distancesInUserTypes[0]), bins)
# usagesHist_nonCommu = np.divide(usagesHist[0], float(np.sum(usagesHist[0])))

# # commuter
# usagesHist = np.histogram(np.array(distancesInUserTypes[1]), bins)
# usagesHist_commu = np.divide(usagesHist[0], float(np.sum(usagesHist[0])))

# bins = np.array(bins[1:])

# fig = plt.figure(figsize=(6, 4))
# ax = plt.subplot(1, 1, 1)

# plt.scatter(bins.tolist(), usagesHist_all.tolist(), marker='o', linewidth=1, facecolor='#41A7D8',
#         edgecolor='k', label='All')
# plt.scatter(bins.tolist(), usagesHist_commu.tolist(), marker='v', linewidth=1, facecolor='#1b7837',
#         edgecolor='k', label='Commuter')
# plt.scatter(bins.tolist(), usagesHist_nonCommu.tolist(), marker='s', linewidth=1, facecolor='#d73027',
#         edgecolor='k', label='Non-commuter')

# plt.xlim(0, maxV)
# plt.ylim(1e-6)
# plt.xticks(np.linspace(0, maxV, numInterval/10+1))
# # plt.xscale("log", nonposy='clip')
# plt.yscale("log", nonposy='clip')
# plt.xlabel(r'Travel displacement, d [km]', fontsize=12)
# plt.ylabel(r"Density, $F(d)$", fontsize=12)
# plt.legend(loc="lower left", frameon=False)

# plt.tight_layout()
# plt.savefig('results/displacement_userType.png', dpi=300)
# plt.savefig('results/displacement_userType.pdf')
# plt.close()


# # travel distance of varying types of trips
# numInterval = 100
# maxV = 100.0
# interval = maxV / numInterval
# bins = np.linspace(0, maxV, numInterval + 1)

# usagesHist = np.histogram(np.array(allTripsDistances), bins)
# usagesHist_all = np.divide(usagesHist[0], float(np.sum(usagesHist[0])))

# # home-work trip
# usagesHist = np.histogram(np.array(distancesInTripTypes[0]), bins)
# usagesHist_HW = np.divide(usagesHist[0], float(np.sum(usagesHist[0])))

# # home-other trip
# usagesHist = np.histogram(np.array(distancesInTripTypes[1]), bins)
# usagesHist_HO = np.divide(usagesHist[0], float(np.sum(usagesHist[0])))

# # work-other trip
# usagesHist = np.histogram(np.array(distancesInTripTypes[2]), bins)
# usagesHist_WO = np.divide(usagesHist[0], float(np.sum(usagesHist[0])))

# # other-other trip
# usagesHist = np.histogram(np.array(distancesInTripTypes[3]), bins)
# usagesHist_OO = np.divide(usagesHist[0], float(np.sum(usagesHist[0])))

# bins = np.array(bins[1:])

# fig = plt.figure(figsize=(6, 4))
# ax = plt.subplot(1, 1, 1)

# plt.scatter(bins.tolist(), usagesHist_all.tolist(), marker='o', linewidth=1, facecolor='#41A7D8',
#         edgecolor='k', label='All')
# plt.scatter(bins.tolist(), usagesHist_HW.tolist(), marker='v', linewidth=1, facecolor='#1b7837',
#         edgecolor='k', label='Home-Work')
# plt.scatter(bins.tolist(), usagesHist_HO.tolist(), marker='s', linewidth=1, facecolor='#d73027',
#         edgecolor='k', label='Home-Other')
# plt.scatter(bins.tolist(), usagesHist_WO.tolist(), marker='s', linewidth=1, facecolor='#762a83',
#         edgecolor='k', label='Work-Other')
# plt.scatter(bins.tolist(), usagesHist_OO.tolist(), marker='s', linewidth=1, facecolor='#b35806',
#         edgecolor='k', label='Other-Other')

# plt.xlim(0, maxV)
# plt.ylim(1e-6)
# plt.xticks(np.linspace(0, maxV, numInterval/10+1))
# # plt.xscale("log", nonposy='clip')
# plt.yscale("log", nonposy='clip')
# plt.xlabel(r'Travel displacement, d [km]', fontsize=12)
# plt.ylabel(r"Density, $F(d)$", fontsize=12)
# plt.legend(loc="lower left", frameon=False)

# plt.tight_layout()
# plt.savefig('results/displacement_tripType.png', dpi=300)
# plt.savefig('results/displacement_tripType.pdf')
# plt.close()

# of commuters : 0.6727500203838562
# of non-commuters : 0.32724997961614377
# of no home : 0.25710860000060926
